In [28]:
import pandas as pd
import numpy as np

# テストデータ作成

正しいブランド名の辞書  
この辞書にある単語との距離を計算し、類似した単語を出力する関数を作成する

In [29]:
brand_list = ["Vuitton", "Gucci"]

brand_list

['Vuitton', 'Gucci']

"item_title"が今回判別対象の商品名  
"Vuitton"と"Gucci"の正しいスペルと間違ったスペルの入った商品名をテストデータとして作成

In [30]:
item_title = ["Pochette voyage Louis Vuitton",
"Pochette voyage Louis Vuiton",
"Pochette voyage Louis Vuittonn",
"Genuine Gucci 1980’s Vintage Navy Blue Boston Bag",
"Genuine Guchi 1980’s Vintage Navy Blue Boston Bag",
"Genuine Gutti 1980’s Vintage Navy Blue Boston Bag",
"Vuittonn and Gutti"
             ]

データフレームにセット  
これが分析対象のデータフレーム

In [31]:
df = pd.DataFrame({
    "id":pd.Series(range(0,len(item_title))),
    "item_title":item_title
})

df

,id,item_title
0,0,Pochette voyage Louis Vuitton
1,1,Pochette voyage Louis Vuiton
2,2,Pochette voyage Louis Vuittonn
3,3,Genuine Gucci 1980’s Vintage Navy Blue Boston Bag
4,4,Genuine Guchi 1980’s Vintage Navy Blue Boston Bag
5,5,Genuine Gutti 1980’s Vintage Navy Blue Boston Bag
6,6,Vuittonn and Gutti


# Levenshtein Distance

In [32]:
import pandas as pd
import numpy as np
from itertools import chain
import Levenshtein

引数  
df："id"と"item_title"をもつデータフレーム  
brand_list：抽出したい正しいブランド名をもつリスト  
dist：距離の閾値。Levenshtein Distanceの場合は値が**小さいほど類似度が高い**

In [33]:
def levenshtein_similarity(df, brand_list, dist_thresh = 0.5):
    # 小文字に揃える
    df["item_title"] = df["item_title"].str.lower()
    brand_list = [c.lower() for c in brand_list]
    
    # 商品名を単語に分割
    title_list = list(set(chain.from_iterable(df["item_title"].str.split(" ").tolist())))

    word_miss = []
    word_corr = []
    word_dist = []

    # 商品名の単語とブランド名の単語を総当りで距離計算
    # 閾値を超えた単語とその距離を格納しておく
    for word_test in title_list:
        for word_dict in brand_list:
                # 文字数で正規化
                dist = Levenshtein.distance(word_test, word_dict)/(max(len(word_test), len(word_dict)))
                # 距離が閾値未満。また、距離が0すなわちブランド名辞書と完全に合致したものは除外。
                if dist < dist_thresh and dist != 0:
                    word_miss.append(word_test)
                    word_corr.append(word_dict)
                    word_dist.append(round(dist, 3))

    result = pd.DataFrame(columns={"id", "item_title"})

    # ミススペルがあった単語と、その単語の正しいスペルと判定されたもの、それらの距離をもつデータフレームを作成
    for m,c,d in zip(word_miss, word_corr, word_dist):
            # スペルミスのあった行を抽出
            temp = df.loc[df["item_title"].str.contains(m), :].copy()
            # 単語と距離をそれぞれのカラムに格納
            temp.loc[temp["item_title"].str.contains(m), "miss_word"] = m
            temp.loc[temp["item_title"].str.contains(m), "corr_word"] = c
            temp.loc[temp["item_title"].str.contains(m), "distance"] = d
            result = result.append(temp, sort = False).sort_values(by = "id")
            result = result.loc[:, ["id", "item_title", "miss_word", "corr_word", "distance"]]

    return result

出力結果  
miss_word：スペルミスと判定された単語  
corr_word：正しいスペルと判定されたブランド名  
distance：levenshtein距離。値が**小さいほど類似している**

In [34]:
levenshtein_similarity(df=df,brand_list=brand_list, dist_thresh=0.5)

,id,item_title,miss_word,corr_word,distance
1,1,pochette voyage louis vuiton,vuiton,vuitton,0.143
2,2,pochette voyage louis vuittonn,vuittonn,vuitton,0.125
4,4,genuine guchi 1980’s vintage navy blue boston bag,guchi,gucci,0.200
5,5,genuine gutti 1980’s vintage navy blue boston bag,gutti,gucci,0.400
6,6,vuittonn and gutti,vuittonn,vuitton,0.125
6,6,vuittonn and gutti,gutti,gucci,0.400


# Jaro-Winkler Distance

In [35]:
import pandas as pd
import numpy as np
from itertools import chain
from pyjarowinkler import distance

引数  
df："id"と"item_title"をもつデータフレーム  
brand_list：抽出したい正しいブランド名をもつリスト  
dist：距離の閾値。Jaro-Winkler Distanceの場合は**値が大きいほど類似度が高い**

In [36]:
def jarowinkler_similarity(df, brand_list, dist_thresh = 0.75):
    # 小文字に揃える
    df["item_title"] = df["item_title"].str.lower()
    brand_list = [c.lower() for c in brand_list]
    
    # 商品名を単語に分割
    title_list = list(set(chain.from_iterable(df["item_title"].str.split(" ").tolist())))

    word_miss = []
    word_corr = []
    word_dist = []

    # 商品名の単語とブランド名の単語を総当りで距離計算
    # 閾値を超えた単語とその距離を格納しておく
    for word_test in title_list:
        for word_dict in brand_list:
                dist =  distance.get_jaro_distance(word_test, word_dict, winkler=True, scaling=0.1)
                # 距離が閾値を超える。また、距離が1すなわちブランド名辞書と完全に合致したものは除外。
                if dist > dist_thresh and dist != 1:
                    word_miss.append(word_test)
                    word_corr.append(word_dict)
                    word_dist.append(round(dist, 3))

    result = pd.DataFrame(columns={"id", "item_title"})

    # ミススペルがあった単語と、その単語の正しいスペルと判定されたもの、それらの距離をもつデータフレームを作成
    for m,c,d in zip(word_miss, word_corr, word_dist):
            # スペルミスのあった行を抽出
            temp = df.loc[df["item_title"].str.contains(m), :].copy()
            # 単語と距離をそれぞれのカラムに格納
            temp.loc[temp["item_title"].str.contains(m), "miss_word"] = m
            temp.loc[temp["item_title"].str.contains(m), "corr_word"] = c
            temp.loc[temp["item_title"].str.contains(m), "distance"] = d
            result = result.append(temp, sort = False).sort_values(by = "id")
            result = result.loc[:, ["id", "item_title", "miss_word", "corr_word", "distance"]]

    return result

出力結果  
miss_word：スペルミスと判定された単語  
corr_word：正しいスペルと判定されたブランド名  
distance：levenshtein距離。値が**大きいほど類似している**

In [37]:
jarowinkler_similarity(df=df, brand_list=brand_list, dist_thresh=0.75)

,id,item_title,miss_word,corr_word,distance
1,1,pochette voyage louis vuiton,vuiton,vuitton,0.97
2,2,pochette voyage louis vuittonn,vuittonn,vuitton,0.98
4,4,genuine guchi 1980’s vintage navy blue boston bag,guchi,gucci,0.91
5,5,genuine gutti 1980’s vintage navy blue boston bag,gutti,gucci,0.79
6,6,vuittonn and gutti,vuittonn,vuitton,0.98
6,6,vuittonn and gutti,gutti,gucci,0.79
